OpenAI API keys: https://platform.openai.com/account/api-keys

In [ ]:
%pip install openai
#https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
%pip install --upgrade tiktoken

# Calculate number of tokens for message

In [ ]:
import tiktoken 
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

In [ ]:
messages = [
  {"role": "system", "content": "¡Hola!"},
]
num_tokens_from_messages(messages)

10

# OpenAI Completion Samples

## Helper functions

In [2]:
import openai
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('config.ini')
openai.api_key = parser.get('OpenAI','API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    #print(response['usage'])
    #print(response.choices[0].finish_reason)
    return response.choices[0].message["content"]

## Ejemplo de explícame el código

In [ ]:
code = f"""
squared_numbers = [(lambda x:x**2)*num) for num in range(1,11)]
"""
prompt = f"""
Explícame el código python delimitado por triple acento \
```{code}```
"""
response = get_completion(prompt)
print(response)

Este código de Python utiliza una expresión lambda para crear una lista de números al cuadrado del 1 al 10. 

La expresión lambda `(lambda x:x**2)` define una función anónima que toma un argumento `x` y devuelve su cuadrado `x**2`. 

Luego, se utiliza una comprensión de lista para aplicar esta función a cada número en el rango del 1 al 10, utilizando la sintaxis `[(lambda x:x**2)*num) for num in range(1,11)]`. 

Esto crea una lista de 10 elementos, donde cada elemento es el resultado de aplicar la función lambda al número correspondiente en el rango del 1 al 10. 

En resumen, el código utiliza una expresión lambda y una comprensión de lista para crear una lista de números al cuadrado del 1 al 10.


# Ejemplo Analizador de reseñas

## Reviews

In [ ]:
# https://www.amazon.es/Wenger-19201-Navaja-suiza/dp/B000R0JDSI/ref=sr_1_1?s=sports&ie=UTF8&qid=1441608629&sr=1-1&keywords=Wenger+-+Navaja+suiza
# https://www.amazon.es/gp/customer-reviews/R24PC48OCI2B40/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000R0JDSI
review_1 = f"""La verdad es que estoy bastante encantado. 
Desde que me llegó, mi vida cambió por completo: 
pude sustituir a mi mujer por la ranura [54], y mi casa por la ranura [115]. 
El frontón [113] es algo pequeño para mi gusto y mi hijo [23] dice que en el colegio [67] no dan Francés. 
A decir verdad, otra cosa que no me ha gustado es que la ranura [67] no viene con un kernel de Linux estable, y la versión de java [98] es la 7 y algunos mods de minecraft no funcionan. 
El lavacoches [203] incorporado no limpia bien el parachoques trasero y me parece que viene poco Uranio [56]. 
En la ranura sorpresa me ha venido la cabeza de Ned Stark, eso ha sido un puntazo.

Lo demás fantástico.
"""

#https://www.amazon.es/gp/customer-reviews/R13BLBLH55IAGZ?ASIN=B01B8R6PF2
review_2 = f"""
¿En serio estás leyendo las opiniones de unas pilas? Son pilas, pilean bien, no hay más que decir. Baja al supermercado, compra un poquito de jamón serrano, un poquito de queso y una botella de vino y deja de leer esto.
"""

#https://www.amazon.es/gp/customer-reviews/R1YL5UM8YQVNJ?ASIN=B000RQU2KU
review_3 = f"""
Es la mejor compra que he hecho nunca. Cuando haga una fiesta y mis amigos vengan a casa podremos pasar toda la noche mirando nuestros móviles y cargarlos a la vez sin dirigirnos la palabra. Y como tengo menos de 9 amigos algunos incluso podremos llevar dos móviles. FIESTÓN.
"""

reviews = [review_1, review_2, review_3]

## Análisis individuales y sencillos de las reviews

In [ ]:

prompt = f"""
¿Cuál es el sentimiento de la reseña de producto, 
que está delimitada por triple acento?

Responda con una sola palabra: "positivo" o "negativo".

Texto a revisar: '''{review_1}'''
"""
sentiment = get_completion(prompt)
print(sentiment)

Positivo.


In [ ]:
prompt = f"""
Crea una lista de las causas por qué la reseña delimitada por triple acento tiene un sentimiento {sentiment}. No incluya más de
cinco elementos en la lista. Escriba su respuesta como una lista de
palabras en minúsculas separadas por comas. 

Texto a revisar: '''{review_1}'''
"""
causes = get_completion(prompt)
print(causes)

casa sustituida, ranura sorpresa, encantado, vida cambiada, puntazo


In [ ]:
prompt = f"""
Crea una lista de las causas por qué la reseña delimitada por triple acento tiene un sentimiento Negativo. No incluya más de
cinco elementos en la lista. Escriba su respuesta como una lista de
palabras en minúsculas separadas por comas. 

Texto a revisar: '''{review_1}'''
"""
causes = get_completion(prompt)
print(causes)

mujer sustituida, casa sustituida, frontón pequeño, colegio sin Francés, versión de java obsoleta.


In [ ]:
prompt = f"""
Crea una lista de emociones del autor de la reseña delimitada por triple acento. No incluya más de
cinco elementos en la lista. Escriba su respuesta como una lista de
palabras en minúsculas separadas por comas.

Texto a revisar: '''{review_1}'''
"""
emotions = get_completion(prompt)
print(emotions)

encantado, cambiado, pequeño, descontento, fantástico


## Análisis de las reviews

In [ ]:
for i in range(len(reviews)):
  prompt = f"""
Dada una reseña delimitada por triple acento.
Tu tarea es crear un json con la siguiente estructura:
- nombre: Resumen valor: Resume la reseña en 1 frase
- nombre: Sentimiento valor: ¿Cuál es el sentimiento de la reseña? Responda con una sola palabra: "positivo" o "negativo" 
- nombre: Causas sentimiento positivo valor: Crea una lista de las causas por qué la reseña tiene un sentimiento positivo. No incluya más de tres elementos en la lista. Escriba su respuesta como una lista de palabras en minúsculas separadas por comas. 
- nombre: Causas sentimiento negativo valor: Crea una lista de las causas por qué la reseña tiene un sentimiento negativo. No incluya más de tres elementos en la lista. Escriba su respuesta como una lista de palabras en minúsculas separadas por comas.
- nombre: Emociones positivas valor: Crea una lista de emociones positivas del autor de la reseña. No incluya más de tres elementos en la lista. Escriba su respuesta como una lista de
palabras en minúsculas separadas por comas.
- nombre: Emociones negativas valor: Crea una lista de emociones negativas positivas del autor de la reseña. No incluya más de tres elementos en la lista. Escriba su respuesta como una lista de
palabras en minúsculas separadas por comas.

reseña: '''{reviews[i]}'''
"""
  result = get_completion(prompt)
  print(f"review_{i} ", result, "\n")

review_0  {
  "Resumen": "Estoy bastante encantado con mi compra",
  "Sentimiento": "positivo",
  "Causas sentimiento positivo": ["sustituir a mi mujer y mi casa por la ranura", "la ranura sorpresa con la cabeza de Ned Stark", "fantástico en general"],
  "Causas sentimiento negativo": ["frontón algo pequeño", "no dan Francés en el colegio", "versión de java no compatible con algunos mods de minecraft"],
  "Emociones positivas": ["encantado", "fantástico", "puntazo"],
  "Emociones negativas": ["pequeña decepción", "insatisfacción", "frustración"]
} 

review_1  {
  "Resumen": "No vale la pena leer la reseña",
  "Sentimiento": "sarcasmo",
  "Causas sentimiento positivo": [],
  "Causas sentimiento negativo": [],
  "Emociones positivas": [],
  "Emociones negativas": ["irritación", "aburrimiento"]
} 

review_2  {
  "Resumen": "La mejor compra que he hecho nunca",
  "Sentimiento": "positivo",
  "Causas sentimiento positivo": ["practicidad", "comodidad", "versatilidad"],
  "Causas sentimiento 